<a href="https://colab.research.google.com/github/GGlivePh/PLF/blob/main/RFID_plus_Camera_IDTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/GGlivePh/PLF

Cloning into 'PLF'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), 105.35 KiB | 2.63 MiB/s, done.


In [ ]:
import os
os.chdir('PLF')

# Advanced Digital Agriculture (DS/AS 875) - Introduction to Python
We will be using the `Jupyter notebook` for many activities this semester. Every notebook has an associated language called the `kernel`. We will be using Python 3 kernel from the IPython project.

In [ ]:
# Install required packages (run in a Colab cell)
!pip install ultralytics opencv-python-headless

# Import necessary libraries
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Simulated BoT-SORT tracker class (simplified version for demonstration)
class SimpleTracker:
    def __init__(self, max_age=30, iou_threshold=0.3):
        self.next_id = 0
        self.tracks = {}  # Dictionary to store track_id and bounding box history
        self.max_age = max_age
        self.iou_threshold = iou_threshold

    def update(self, detections):
        """
        Update tracks based on detections. For simplicity, we assume detections
        come as a list of bounding boxes [x1, y1, x2, y2] and confidence scores.
        """
        updated_tracks = {}
        # A naive association: each detection gets a new track ID if no match is found
        for det in detections:
            # Here, you would implement a proper association (e.g., Kalman filter + Hungarian algorithm)
            updated_tracks[self.next_id] = det
            self.next_id += 1
        self.tracks = updated_tracks
        return self.tracks

# Simulated RFID reader function
def get_rfid_reading(frame, feeder_region):
    """
    Simulated RFID reading: if a detection's center falls within feeder_region, return an animal ID.
    feeder_region: a tuple (x_min, y_min, x_max, y_max)
    For demonstration, we randomly assign an ID from a fixed list.
    """
    # Simulated RFID dictionary (in practice, this reading would come from a DB or sensor)
    simulated_rfid = {'animal_ids': [7926, 7940, 7993, 7992, 8003, 7909, 7911]}
    # For this example, we simply return a random animal ID if a condition is met
    # In real code, you'll check for detections within the feeder area and read the RFID accordingly.
    return np.random.choice(simulated_rfid['animal_ids'])

# Define the feeder region (example values in pixels)
feeder_region = (100, 100, 300, 300)  # (x_min, y_min, x_max, y_max)

# Load the finetuned YOLOv8 model (replace 'yolov8n.pt' with the appropriate custom model if available)
model = YOLO('yolov8n.pt')  # Use the tiny or small model for quick testing

# Initialize the tracker
tracker = SimpleTracker()

# Open video file or stream; for demonstration, we assume a sample video file 'pig_video.mp4'
video_path = 'pig_video.mp4'
cap = cv2.VideoCapture(video_path)

# Dictionary to store track_id to animal_id mapping (RFID linking)
track_to_animal_id = {}

# Process video frames
frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_idx += 1

    # Run detection on the frame using YOLOv8
    results = model(frame)[0]  # Assuming the result is the first element of the model inference output
    # Extract bounding boxes, confidences, and optionally classes (format: [x1, y1, x2, y2, confidence, class])
    detections = []
    for result in results.boxes.data.cpu().numpy():
        x1, y1, x2, y2, conf, cls = result
        detections.append([x1, y1, x2, y2, conf])

    # Update tracker with current detections; this returns a mapping {track_id: detection_bbox}
    tracks = tracker.update(detections)

    # Check if any track’s detection falls within the feeder region to perform RFID linking
    for track_id, bbox in tracks.items():
        x1, y1, x2, y2, conf = bbox
        cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
        # Check if the center is within feeder region
        if feeder_region[0] <= cx <= feeder_region[2] and feeder_region[1] <= cy <= feeder_region[3]:
            # Get RFID reading (simulate reading from the feeder)
            animal_id = get_rfid_reading(frame, feeder_region)
            # Link this RFID animal ID with the current track_id
            track_to_animal_id[track_id] = animal_id

    # Optionally, visualize tracking results
    for track_id, bbox in tracks.items():
        x1, y1, x2, y2, conf = bbox
        # Draw bounding box and display track id and if available, the linked animal id
        label = f"ID: {track_id}"
        if track_id in track_to_animal_id:
            label += f" (Animal: {track_to_animal_id[track_id]})"
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
        cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

    # Display every 100 frames (for debugging purposes)
    if frame_idx % 100 == 0:
        plt.figure(figsize=(8,6))
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.title(f"Frame {frame_idx}")
        plt.axis("off")
        plt.show()

cap.release()

# After processing the video, you can combine track segments for each animal,
# and compute metrics such as total tracking time per animal.
print("RFID linking (track ID -> animal ID):")
print(track_to_animal_id)


0.09653235263005393

# Table of Contents
* [1. Python data type](#1.-Python-data-type)
    * [1.1 Numbers](#1.1-Numbers)
    * [1.2 String](#1.2-String)
    * [1.3 Boolean](#1.3-Boolean)
* [2. Python operators](#2.-Python-operators)
* [3. Python comparison operators](#3.-Python-comparison-operators)
    * [3.1 Equal](#3.1-Equal)
    * [3.2 Not equal](#3.2-Not-equal)
    * [3.3 Greater than](#3.3-Greater-than)
    * [3.4 Less than](#3.4-Less-than)
    * [3.5 Greater than or equal to](#3.5-Greater-than-or-equal-to)
    * [3.6 Less than or equal to](#3.6-Less-than-or-equal-to)
* [4. Print](#4.-Print)
* [5. Access items or index](#5.-Access-items-or-index)
* [6. Lists, dictionaries, and tuples](#6.-Lists,-dictionaries,-and-tuples)
    * [6.1 List](#6.1-List)
    * [6.2 Nest lists](#6.2-Nest-lists)
    * [6.3 Dictionaries](#6.3-Dictionaries)
    * [6.4 Tuple](#6.4-Tuple)
* [7. Control flow statements](#7.-Control-flow-statements)
    * [7.1 *if, elif, else* statements](#7.1-if,-elif,-else-statements)
    * [7.2 *for* loop](#7.2-for-loop)
    * [7.3 *while* loop](#7.3-while-loop)
* [8. Python functions](#8.-Python-functions)
    * [9.1. Creating a function](#8.1-Creating-a-function)
    * [9.2. Calling a function](#8.2-Calling-a-function)
* [9. Read and write data](#9.-Read-and-write-data)
    * [10.1 Read](#9.1-Read)
    * [10.2 Write](#9.2-Write)
* [10. Read and write images](#10.-Read-and-write-images)
    * [11.1 Read](#10.1-Read)
    * [11.2 Visualize image](#10.2-Visualize-image)
    * [11.3 Write](#10.3-Write)
* [11. References](#11.-References)

# 1. Python data type

## 1.1 Numbers

In [ ]:
x = 34      # Integer
x

In [ ]:
y = 3.45    # Numeric or Floats
y

## 1.2 String

In [ ]:
z = 'Hello'
z

In [ ]:
w = 'This is a string'
w

In [ ]:
c = z + "! " + w
c

## 1.3 Boolean

In [ ]:
a = True
b = False

# 2. Python operators
Operators +, -, * and / work just like in most other languages

In [ ]:
2 + 2  # addition

In [ ]:
10 - 5 # subtraction

In [ ]:
5 * 6  # multiplication

In [ ]:
8 / 5  # division

In [ ]:
5 ** 2 # Exponentiation

# 3. Python comparison operators
Comparison operators are used to compare two values.

## 3.1 Equal

In [ ]:
x == y

## 3.2 Not equal

In [ ]:
x != y

## 3.3 Greater than

In [ ]:
x > y

## 3.4 Less than

In [ ]:
x < y

## 3.5 Greater than or equal to

In [ ]:
x >= y

## 3.6 Less than or equal to

In [ ]:
x <= y

# 4. Print
Evaluating and display result as an `output`, versus evaluating and printing result (side-effect).

In [ ]:
print(x)

In [ ]:
print('Hello World!')

In [ ]:
print("""\
Usage: intro [OPTIONS]
     -h                        Display this usage message
     -H hostname               Hostname to connect to
""")

# 5. Access items or index
One way to remember how slice works
<pre>
+---+---+---+---+---+---+
| P | y | t | h | o | n |
+---+---+---+---+---+---+
  0   1   2   3   4   5
</pre>

In [ ]:
word = 'Python'

In [ ]:
word[0] # character in position 0

In [ ]:
word[5] # character in position 5

In [ ]:
word[0:2] # characters from position 0 (included) to 2 (excluded)

In [ ]:
word[:2] + word[2:] # start and end always excluded -> s[:i] + s[i:]

# 6. Lists, dictionaries, and tuples

Python has three very useful data structures built into the language:
* lists: [1,2,..]
* dictionaries (hash tables): {"Name": "Joao"}
* tuples: (item, ...)

## 6.1 List
Lists are used to store multiple items in a single variable.

In [ ]:
numbers = [1, 4, 9, 16, 25]

In [ ]:
numbers[0] # indexing returns the item

In [ ]:
numbers[3:]

In [ ]:
numbers + [64, 72, 90,104] # concatenation

In [ ]:
numbers[3] = 200 # replace a value in the list
numbers

In [ ]:
numbers.append(100000)
numbers

In [ ]:
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
letters

In [ ]:
letters[2:5] = ['C', 'D', 'E'] # replace letters
letters

In [ ]:
letters[2:5] = [] # remove them
letters

In [ ]:
len(letters)

In [ ]:
letters[:] = [] # clear the list by replacing all the elements with an empty list
letters

## 6.2 Nested lists

In [ ]:
a = ['a', 'b', 'c']
n = [1, 2, 3]

In [ ]:
x = [a, n]
x

In [ ]:
x[0]

In [ ]:
x[0][1]

##  6.3 Dictionaries
Dictionaries are used to store data values in `key:value pairs`. A dictionary is a collection which is ordered*, changeable and do not allow duplicates.

In [ ]:
mydict = {
  "CowID": "8876",
  "Farmer": "Joao",
  "Birth": 1964,
  "Weight": [1800, 1910, 1730]
}

In [ ]:
print(len(mydict))

In [ ]:
print(mydict)

In [ ]:
print(mydict["CowID"])

In [ ]:
print(mydict["CowID"], mydict["Birth"])

In [ ]:
print("CowID: " + mydict["CowID"] + "\n" +
      "Farmer Name: " + mydict["Farmer"] + "\n" +
      "Body Weight: " + str(mydict["Weight"][1]))

## 6.4 Tuples
Tuples are used to store multiple items in a single variable which is ordered and unchangeable.

In [ ]:
tuple1 = ("apple", "banana", "cherry")

In [ ]:
print(tuple1)

In [ ]:
tuple2 = (200, 200, 3)

In [ ]:
print(tuple2)

# 7. Control flow statements

### 7.1 *if, elif, else* statements
Perhaps the most well-known statement type is the if statement.
There can be zero or more elif parts, and the else part is optional. The keyword ‘elif’ is short for ‘else if’, and is useful to avoid excessive indentation.

In [ ]:
x = 10
if x <= 10:
    print('The integer is lower than TEN!')

In [ ]:
x = 10
if x <= 10:
    print('The integer is lower than TEN!')
else:
    print('The integer is greater than TEN!')

In [ ]:
x = int(input("Please enter an integer from 1-100: "))
if x < 0:
    print('The integer is greater than ZERO!')
elif x >= 0 and x < 30:
    print('The integer is greater than ZERO and lower than 30!')
elif x >= 30 and x < 70:
    print('The integer is greater than 30 and lower than 70!')
else:
    print('The integer is greater than 70!')

## 7.2 *for* loop
A `for` loop is used for iterating over the items of any sequence (a list or a string), in the order that they appear in the sequence.

In [ ]:
# Measure some strings:
words = ['cat', 'dog', 'cow', 'horse']
for w in words:
    print(w)

## 7.3 *while* loop
With the `while` loop we can execute a set of statements as long as a condition is true.

In [ ]:
n = 1
while n <= 5:
    print('Number: ' + str(n))
    n+=1

# 8. Python functions
A function is a block of code which only runs when it is called.
* You can pass data, known as parameters, into a function.
* A function can return data as a result.
* If you don't specify a return value, then it will default to returning None.

## 8.1 Creating a function

In [ ]:
def plus(a, b):
    x = a + b
    return x

In [ ]:
plus(3, 4)

## 8.2 Calling a function

In [ ]:
import operator

# 9. Read and write data

## 9.1 Read

In [ ]:
import os
import pandas as pd

In [ ]:
data1 = pd.read_csv('data.csv')

In [ ]:
data1

In [ ]:
data2 = pd.read_table('data.csv', delimiter=',')

In [ ]:
data2

## 9.2 Write

In [ ]:
data1.to_csv('dataNew.csv', sep=',', header = True, index = False)

In [ ]:
os.listdir('.') # list files

In [ ]:
data1.to_csv('dataNew.txt', sep=' ', header = True, index = False)

In [ ]:
os.listdir('.') # list files

# 10. Read and write images

## 10.1 Read


In [ ]:
from skimage.io import imread, imshow, imsave

In [ ]:
image = imread(r'cow.jpg')

In [ ]:
image.shape

## 10.2 Visualize image

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
im = plt.imshow(image)

## 10.3 Write

In [ ]:
imsave("cowNew.png", image)

In [ ]:
os.listdir('.') # list files

# 11. References
* https://www.python.org/
* https://www.w3schools.com/python/default.asp